In [23]:
import os
import pathlib
import pandas as pd
import numpy as np
import glob
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.SeqIO.FastaIO import SimpleFastaParser
import subprocess 
from BCBio import GFF
import re 
import matplotlib.pyplot as plt 

In [24]:

datapath = "./data/*"
datadep_path =  "./data/datadep/*"
path_input = "/Users/bustin/Desktop/capstone/MOSSBIOINFO/transcriptomics/march25/biojustin/data/datadep"
test_path = "/Users/bustin/Desktop/capstone/MOSSBIOINFO/transcriptomics/march25/biojustin/data/datatest"
os.chdir("/Users/bustin/Desktop/capstone/MOSSBIOINFO/transcriptomics/march25")

In [26]:
#Fasta file writer 
def FastaFile(gene_id,seq,outname):
    initial_path = os.getcwd()
    with open(outname,'a') as f:
        f.write(">"+gene_id+"\n"+seq+"\n") 
    os.chdir(initial_path)


def extract_upstream_sequence(contig_id, start, end,genome_dict, upstream_length=3000):
    """
    Extracts the sequence upstream of a gene given its genomic location.
    contig_id: str, ID of the contig containing the gene.
    start: int, 1-based start position of the gene.
    end: int, 1-based end position of the gene.
    upstream_length: int, length of sequence to extract upstream of the gene.
    Returns: str, upstream sequence of the gene.
    """

    # Get sequence object for contig
    contig_seq = genome_dict[contig_id].seq
    # Compute start and end positions of upstream sequence
    upstream_start = max(start - upstream_length, 0)
    upstream_end = start - 1
    # Extract upstream sequence
    upstream_seq = contig_seq[upstream_start:upstream_end]
    return str(upstream_seq)

In [30]:
#Class Class()
class Class():
    def load(self,filename,path_input=path_input): #load proteome data (single or multiple fasta files) 
        os.chdir(path_input)
        # Create an empty DataFrame with the desired column names
        self.input_seq_db  = pd.DataFrame(columns=["gene_id", "seq", "len"])
        # Open the FASTA file and iterate over its records
        with open(filename) as in_handle:
            for title, seq in SimpleFastaParser(in_handle):
                # Add the current record to the DataFrame
                len_seq = len(seq)
                gene_id = title.split()[0]
                self.input_seq_db = pd.concat([self.input_seq_db, pd.DataFrame([[gene_id, seq, len_seq]], columns=["gene_id", "seq", "len"])])
        self.input_seq_db.set_index(keys=self.input_seq_db["gene_id"],inplace=True)
        return self 
    def combine_df(self,filename,path_input=path_input): #load proteome data (single or multiple fasta files) 
        os.chdir(path_input)
        with open(filename) as in_handle:
            for title, seq in SimpleFastaParser(in_handle):
                # Add the current record to the DataFrame
                len_seq = len(seq)
                gene_id = title.split()[0]
                self.input_seq_db = pd.concat([self.input_seq_db, pd.DataFrame([[gene_id, seq, len_seq]], columns=["gene_id", "seq", "len"])])
        return self 

    def hmmer_cyp(self,input_fasta,path_input):
        os.chdir(path_input)
        # Run HMMER
        subprocess.check_output(['hmmsearch', "-E","0.1", "--tblout", "hmm_hits.txt","Pfam-A.hmm",input_fasta])
        # Process the HMMER output and add hits to the DataFrame
        with open("hmm_hits.txt") as f:
            for line in f:
                if not line.startswith('#'):
                    fields = line.strip().split()
                    e_value = fields[4]
                    gene_id = fields[0]
                    domain_name = fields[3]
                    if domain_name in ['PF00067.25', 'PF00067']:
                        mask = self.input_seq_db["gene_id"] == gene_id
                        if mask.any():
                            self.input_seq_db.loc[mask, "Hmmer_CYP"] = e_value
        # Remove the HMMER output file
        os.remove("hmm_hits.txt")
        return self 
    def hmmer_tps(self,input_fasta,path_input):
        os.chdir(path_input)
        # Run HMMER
        subprocess.check_output(['hmmsearch', "-E","0.1", "--tblout", "hmm_hits.txt","Pfam-A.hmm",input_fasta])
        # Process the HMMER output and add hits to the DataFrame
        with open("hmm_hits.txt") as f:
            for line in f:
                if not line.startswith('#'):
                    fields = line.strip().split()
                    e_value = fields[4]
                    gene_id = fields[0]
                    domain_name = fields[3]
                    if domain_name in ['PF01397.24','PF03936.19','PF19086.3']:
                        mask = self.input_seq_db["gene_id"] == gene_id
                        if mask.any():
                            self.input_seq_db.loc[mask, "Hmmer_TPS"] = e_value
        # Remove the HMMER output file
        os.remove("hmm_hits.txt")
        return self 
 
    def blastp_comparison(self, query, subject):
        # Run blastp
        subprocess.check_output(['blastp', "-query", query,"-subject",subject, "-out","blastp_output.txt","-outfmt","6","-max_target_seqs","1","-evalue","0.1"])
        # Process the blastp output and add hits to the DataFrame
        with open('blastp_output.txt') as f:
            for line in f:
                fields = line.split()
                gene_id = fields[0]
                subject_id = fields[1]
                percentage = fields[2]
                e_value = fields[10]
                mask = self.input_seq_db["gene_id"] == gene_id
                header = subject_id+"aa identity"
                if mask.any():
                    self.input_seq_db.loc[mask, header] = str(percentage)
                    self.input_seq_db.loc[mask, "evalue"] = str(e_value)
        os.remove("blastp_output.txt")
        return self
    
    def blastp_label(self,query,subject):
        # Run blastp of the transcriptome object against a target protein, label the e-value and %homology of hits 
        subprocess.check_output(['blastp', "-query", query,"-subject",subject, "-out","blastp_output.txt","-outfmt","6","-max_target_seqs","1","-evalue","0.1"])
        # Process the blastp output and add hits to the DataFrame
        with open('blastp_output.txt') as f:
            for line in f:
                fields = line.split()
                gene_id = fields[0]
                subject_id = fields[1]
                percentage = fields[2]
                e_value = fields[10]
                mask = self.input_seq_db["gene_id"] == gene_id
                if mask.any():
                    self.input_seq_db.loc[mask, "CYP blastp - CYP Name"] = subject_id
                    self.input_seq_db.loc[mask, "CYP blastp - aa perc"] = percentage
                    self.input_seq_db.loc[mask, "CYP blastp - e value"] = e_value
        os.remove("blastp_output.txt")
        return self  
    
    def get_distribution_identity(self,header_name):
        #Use this after using blastp_label to plot the results 
        plt.hist(self.input_seq_db[header_name].str.split(",").str[0].astype(float), bins=20, range=(0, 100))
        plt.xlabel("Percentage Amino Acid Identity")
        plt.ylabel("Count")
        plt.title("Distribution of Percentage Amino Acid Identity")
        plt.show()
        return plt 
    
    def EC_CLEAN(self,input_file ): 
        
        #First ensure you deactivate conda refresh the terminal to only .venv 
        #path_ec = "/Users/bustin/Desktop/capstone/MOSSBIOINFO/transcriptomics/march25/CLEAN-main/app"
        #os.chdir(path_ec)
        #script_path = "/Users/bustin/Desktop/capstone/MOSSBIOINFO/transcriptomics/march25/CLEAN-main/clean_script.sh"
        #subprocess.check_output(["sh","-c",script_path],input=b"Y\n")
    
        #Run the tool 
        input_file_1 = input_file.strip(".fasta")
        subprocess.check_output(['python', "CLEAN_infer_fasta.py", "--fasta_data",input_file_1])
        # Read in CSV result file and keep only the first two columns
        output_path = "/Users/bustin/Desktop/capstone/MOSSBIOINFO/transcriptomics/march25/CLEAN-main/app/results/inputs"
        os.chdir(output_path)
        output_file = input_file.strip(".fasta") + "_maxsep.csv"
        results_ec = pd.read_csv(output_file, usecols=[0, 1], error_bad_lines=False,header=None)
        self.input_seq_db = self.input_seq_db.reset_index(drop=False)
        self.input_seq_db["EC_Label(code, cluster distance)"] = results_ec.iloc[:,1]
        return self 

    def locate_genome(self,filename): #to be worked on
        with open(filename) as in_handle:
            for title, seq in SimpleFastaParser(in_handle):
                # Add the current record to the DataFrame
                len_seq = len(seq)
                gene_id = title.split()[0]
                location = title.split()[4]
                a= location.strip(":").split("-")
                start = a[0]
                end = a[1]
                contig = title.split()[3]
                contig = contig.split("=")[1].strip(":")
                self.input_seq_db.at[gene_id,"contig"] = contig
                self.input_seq_db.at[gene_id,"location"] = str(start)+","+str(end) 
        return self 
    
    def get_promoter(self,genome_file):#to be worked on
        genome_dict = SeqIO.to_dict(SeqIO.parse(genome_file, "fasta"))
        for gene_id in self.input_seq_db["gene_id"]:
            location = self.input_seq_db.at[gene_id,"location"]
            contig = self.input_seq_db.at[gene_id,"contig"]
            sequence_genome = genome_dict[contig].seq
            b = location.split(",")
            start = int(b[0])
            end = int(b[1])
            print(start, end)
            print(type(start), type(end))
            #start = int(location[0])
            #end = int(location[1])
            upstream_seq = extract_upstream_sequence(contig, start, end,genome_dict=genome_dict)
            wbox_regex = re.compile("TTGACT|TTGACC")
            has_wbox = bool(wbox_regex.search(upstream_seq))
            self.input_seq_db.at[gene_id,"promoter"] = upstream_seq
            self.input_seq_db.at[gene_id,"wbox?"] = has_wbox
                    #promoter_seq = self.input_seq_db["promoter"] 

In [33]:
tacca = Class()
tacca.load("tacca_trim_assembly.fasta")
with open("hmm_hits.txt") as f:
      for line in f:
        if not line.startswith('#'):
            fields = line.strip().split()
            e_value = fields[4]
            gene_id = fields[0]
            domain_name = fields[3]
            if domain_name in ['PF00067.25', 'PF00067']:
                mask = tacca.input_seq_db["gene_id"] == gene_id
                if mask.any():
                    tacca.input_seq_db.loc[mask, "Hmmer_CYP"] = e_value
tacca.input_seq_db.to_csv("tchantrieri_cyps_.csv")

In [60]:
#BTS1 PROJECT
bts_path = "/Users/bustin/Desktop/capstone/MOSSBIOINFO/transcriptomics/march25/biojustin/data/datadep/bts1_proj" 
os.chdir(bts_path)
filelist = os.listdir(os.curdir)
filelist.pop(2) 
print(filelist)

uniprotkb_gpps = BioBustin() 
uniprotkb_gpps.load(filelist[0],path_input=bts_path) 

['uniprotkb_gpps.fasta', 'uniprotkb_fpps.fasta', 'ncbi_bts1.fasta', 'uniprotkb_bts1.fasta', 'ncbi_ggpps.fasta', 'uniprotkb_ggpps.fasta', 'ncbi_fpps.fasta', 'ncbi_gpps.fasta']


In [ ]:
tacca = BioBustin() 
tacca.load("tchantrieri_proteome.fasta")
path_ec = "/Users/bustin/Desktop/capstone/MOSSBIOINFO/transcriptomics/march25/CLEAN-main/app"
os.chdir(path_ec)
tacca.EC_CLEAN(input_file="tchantrieri_proteome.fasta")




In [29]:
tacca = Class() 
tacca.load("tchantrieri_proteome.fasta")

with open("hmm_hits_tps.txt") as f:
    for line in f:
        if not line.startswith('#'):
            fields = line.strip().split()
            e_value = fields[4]
            gene_id = fields[0]
            domain_name = fields[3]
            if domain_name in ['PF00432.24']:
                mask = tacca.input_seq_db["gene_id"] == gene_id
                if mask.any():
                    tacca.input_seq_db.loc[mask, "Hmmer_prenyltr"] = e_value

In [30]:
tacca.input_seq_db.to_csv("tchantrieri_prenyltr_unfiltered.csv")

In [17]:
tacca.input_seq_db.to_csv("tchantrieri_tps3.csv",index=False)

In [13]:
tacca.input_seq_db.to_csv("tchantrieri_tps_unfiltered.csv", index=False)

In [ ]:
import requests
from bs4 import BeautifulSoup

# Define a function that checks whether there are SRA experiments available for a given taxon
def check_sra(txid, seq_type):
    url = f"https://www.ncbi.nlm.nih.gov/sra?term={txid}%20{seq_type}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    title = soup.find("title").text
    print(title)
    if "No items found" in title:
        return "No"
    else: 
        print('hi')
        return "Yes"


# Read in the CSV file and specify "Plant Taxonomy" as the column label
df = pd.read_csv('unique_plant_species.csv')
print(df)

# Add a new column for DNA-seq experiments and populate it with "Yes" or "No"
df['DNA-seq'] = df['Plant Species'].apply(lambda x: check_sra(x, "DNA"))

# Add a new column for RNA-seq experiments and populate it with "Yes" or "No"
df['RNA-seq'] = df['Plant Species'].apply(lambda x: check_sra(x, "RNA"))

# Save the updated DataFrame to a new CSV file
df.to_csv('sra_experiments.csv', index=False)

In [ ]:
# Set filenames
genome_fasta = "cplumiforme_genome.fasta"
gff_file = "cplumiforme_gff.txt"
output_bed = "visualization.bed"
output_pdf = "genes.pdf"


# Filter GFF to only include genes in list
from Bio import SeqIO
import matplotlib.pyplot as plt
import numpy as np
# Load genome fasta file and create a dictionary of sequence IDs and sequences
genome_dict = SeqIO.to_dict(SeqIO.parse(genome_fasta, "fasta"))

# Define a function to plot a gene
def plot_gene(start, end, seq):
    # Get the gene coordinates
    gene_start = min(start, end)
    gene_end = max(start, end)

    # Convert the sequence to a list of bytes
    seq_list = list(seq)

    # Create a mask for the gene
    gene_mask = np.zeros(len(seq_list))
    gene_mask[gene_start:gene_end] = 1

    # Create a plot of the sequence and the gene mask
    fig, ax = plt.subplots(figsize=(10, 2))
    sequence_list = np.frombuffer(b''.join(seq.encode() for seq in seq_list), dtype=np.uint8)
    ax.imshow(np.array([gene_mask, np.zeros(len(sequence_list)), np.zeros(len(sequence_list))]).T,
              aspect="auto")
    ax.imshow(np.array([np.zeros(len(sequence_list)), np.zeros(len(sequence_list)), sequence_list == b'G']).T,
              aspect="auto", cmap="gray_r")
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlim(0, len(sequence_list))
    ax.set_ylim(-0.5, 2.5)

# Iterate over rows and plot each gene
for i, row in test.input_seq_db.iloc[0:92].iterrows():
    gene_id = row["gene_id"]
    print(gene_id)
    gene_pos = row["Genomic Location"]
    start = int(gene_pos[0])
    end = int(gene_pos[1])
    seq = row["seq"] 
    plot_gene(start=start,end=end,seq=seq)

In [ ]:
#BLASTING AGAINST NELSON MOSS DB 
path_input = "/Users/bustin/Desktop/capstone/MOSSBIOINFO/transcriptomics/march25/biojustin/data/datadep"
os.chdir(path_input)
nelson = BioBustin()
nelson.load(filename="cyp_filtered_6000aa_wref.fasta",path_input=path_input)
for index,row in nelson.input_seq_db.iterrows():
    seq = row["seq"]
    query = row["gene_id"]
    outname = subject+".fasta"
    FastaFile(gene_id=query,seq=seq,outname= outname)
    nelson.blastp_label(query=outname, subject="cyp_db_full.fasta")
    os.remove(outname)


In [ ]:
nelson.input_seq_db.loc[:,"CYP blastp - CYP Name":"CYP blastp - e value]"].to_csv("nelson_label_6000.csv")

In [ ]:
path_input = "/Users/bustin/Desktop/capstone/MOSSBIOINFO/transcriptomics/march25/biojustin/data/datadep"

test1 = BioBustin()
test1.load(filename="cyp_filtered_6000aa_wref.fasta",path_input=path_input)
test1.hmmer_cyp(pfam="PF00067.hmm",input_fasta="cyp_filtered_6000aa_wref.fasta",path_input=path_input)
test1.blastp_comparison(query="cyp_filtered_6000aa_wref.fasta",subject="CpCYP964A1_p.fasta")
test1.EC_CLEAN(input_file="cyp_filtered_6000aa_wref.fasta")
print(test1.input_seq_db)

In [ ]:
#Make the full nelson cyp database fasta files 
with open("cyp_db_full.fasta",'a') as f:
    for i in ["Chlorophyte_algae","Charophyte_algae","MossRaw","Liverworts","Hornworts","Lycophytes","Fern","Gymnosperm"]:
        df = pd.read_excel("nelson_cyp_db.xlsx", i)
        for index, row in df.iterrows():
            seq = row["Sequence"]
            gene_id = str(row["CYP name"]) + "," + str(row["Species"]) + "," + i
            f.write(">"+gene_id+"\n"+seq+"\n")


In [ ]:
#Make just moss db 
xls = pd.ExcelFile('NELSON_DB.xlsx')
df = pd.read_excel(xls, "MossRaw")
with open("cyp_db_moss.fasta",'w') as f:
    for index, row in df.iterrows():
        seq = row["Sequence"]
        gene_id = row["CYP name"] + "," + row["Species"] + "," + "MossRaw"
        f.write(">"+gene_id+"\n"+seq+"\n")
        


In [ ]:
#OS STUFF PATH STUFF 
#print(os.getcwd())
#fasta_file = "CpCYP964A1_p.fasta"
#print(os.path.isfile(fasta_file))
#st = os.stat(fasta_file)
#print(st)